# Analyzing the Graph of Thrones with Neo4j

In [5]:
#! pip install py2neo
from py2neo import Graph
graph = Graph()

In [3]:
!pip install py2neo --upgrade

    100% |████████████████████████████████| 143kB 3.0MB/s 
  Found existing installation: py2neo 2.0.8
    Uninstalling py2neo-2.0.8:
      Successfully uninstalled py2neo-2.0.8


## Import into Neo4j


In [7]:
graph.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")

for record in graph.run('''
LOAD CSV WITH HEADERS FROM "https://www.macalester.edu/~abeverid/data/stormofswords.csv" AS row
MERGE (src:Character {name: row.Source})
MERGE (tgt:Character {name: row.Target})
MERGE (src)-[r:INTERACTS]->(tgt)
SET r.weight = toInt(row.Weight)
RETURN count(*) AS paths_written
'''):
    print(record)

('paths_written': 352)


In [1]:
for r in graph.run('''
MATCH p=(:Character)-[:INTERACTS]-(:Character)
RETURN p
'''):
    print(r)

![](http://www.lyonwj.com/public/img/got-graph-full.png)

## Analyzing the network

### Number of characters

In [9]:
for record in graph.run("MATCH (c:Character) RETURN count(c) AS num"):
    print(record)

('num': 107)


### Summary statistics

In [10]:
for record in graph.run('''
MATCH (c:Character)-[:INTERACTS]->()
WITH c, count(*) AS num
RETURN min(num) AS min, max(num) AS max, avg(num) AS avg_characters, stdev(num) AS stdev
'''):
    print(record)

('min': 1, 'max': 24, 'avg_characters': 4.957746478873239, 'stdev': 6.227672391875085)


### Diameter of the network

In [54]:
for r in graph.run('''
// Find maximum diameter of network
// maximum shortest path between two nodes
MATCH (a:Character), (b:Character) WHERE id(a) > id(b)
MATCH p=shortestPath((a)-[:INTERACTS*]-(b))
RETURN length(p) AS len, extract(x IN nodes(p) | x.name) AS path
ORDER BY len DESC LIMIT 4
'''):
    print(r)

('len': 6, 'path': ['Illyrio', 'Belwas', 'Daenerys', 'Robert', 'Tywin', 'Oberyn', 'Amory'])
('len': 6, 'path': ['Illyrio', 'Belwas', 'Daenerys', 'Robert', 'Sansa', 'Bran', 'Jojen'])
('len': 6, 'path': ['Illyrio', 'Belwas', 'Daenerys', 'Robert', 'Stannis', 'Davos', 'Shireen'])
('len': 6, 'path': ['Illyrio', 'Belwas', 'Daenerys', 'Robert', 'Sansa', 'Bran', 'Luwin'])


### Shortest path

In [55]:
for r in graph.run('''
// Shortest path from Catelyn Stark to Khal Drogo
MATCH (catelyn:Character {name: "Catelyn"}), (drogo:Character {name: "Drogo"})
MATCH p=shortestPath((catelyn)-[INTERACTS*]-(drogo))
RETURN p
'''):
    print(r)

('p': (e95a2a2)-[:INTERACTS {weight:19}]->(ec0683e)-[:INTERACTS {weight:4}]->(b5ccf71)<-[:INTERACTS {weight:11}]-(b8220b2)-[:INTERACTS {weight:6}]->(d354c17))


![](http://www.lyonwj.com/public/img/shortestpath-got.png)

### All shortest paths

In [58]:
for r in graph.run('''
// All shortest paths from Catelyn Stark to Khal Drogo
MATCH (catelyn:Character {name: "Catelyn"}), (drogo:Character {name: "Drogo"})
MATCH p=allShortestPaths((catelyn)-[INTERACTS*]-(drogo))
RETURN p
'''):
    print(r)

![](http://www.lyonwj.com/public/img/allshortestpaths-got.png)

### Pivotal nodes

In [59]:
for r in graph.run('''
// Find all pivotal nodes in network
MATCH (a:Character), (b:Character)
MATCH p=allShortestPaths((a)-[:INTERACTS*]-(b)) WITH collect(p) AS paths, a, b
MATCH (c:Character) WHERE all(x IN paths WHERE c IN nodes(x)) AND NOT c IN [a,b]
RETURN a.name, b.name, c.name AS PivotalNode SKIP 490 LIMIT 10
'''):
    print(r)

('a.name': 'Aegon', 'b.name': 'Thoros', 'PivotalNode': 'Daenerys')
('a.name': 'Aegon', 'b.name': 'Thoros', 'PivotalNode': 'Robert')
('a.name': 'Drogo', 'b.name': 'Ramsay', 'PivotalNode': 'Robb')
('a.name': 'Styr', 'b.name': 'Daario', 'PivotalNode': 'Daenerys')
('a.name': 'Styr', 'b.name': 'Daario', 'PivotalNode': 'Jon')
('a.name': 'Styr', 'b.name': 'Daario', 'PivotalNode': 'Robert')
('a.name': 'Qhorin', 'b.name': 'Podrick', 'PivotalNode': 'Jon')
('a.name': 'Qhorin', 'b.name': 'Podrick', 'PivotalNode': 'Sansa')
('a.name': 'Orell', 'b.name': 'Theon', 'PivotalNode': 'Jon')
('a.name': 'Illyrio', 'b.name': 'Bronn', 'PivotalNode': 'Belwas')


In [61]:
for r in graph.run('''
MATCH (a:Character {name: "Drogo"}), (b:Character {name: "Ramsay"})
MATCH p=allShortestPaths((a)-[:INTERACTS*]-(b))
RETURN p
'''):
    print(r)

![](http://www.lyonwj.com/public/img/pivotal-path.png)

## Centrality measures

### Degree centrality

In [64]:
for r in graph.run('''
MATCH (c:Character)-[:INTERACTS]-()
RETURN c.name AS character, count(*) AS degree ORDER BY degree DESC LIMIT 10
'''):
    print(r)

('character': 'Tyrion', 'degree': 36)
('character': 'Jon', 'degree': 26)
('character': 'Sansa', 'degree': 26)
('character': 'Robb', 'degree': 25)
('character': 'Jaime', 'degree': 24)
('character': 'Tywin', 'degree': 22)
('character': 'Cersei', 'degree': 20)
('character': 'Arya', 'degree': 19)
('character': 'Joffrey', 'degree': 18)
('character': 'Robert', 'degree': 18)


### Weighted degree centrality

In [65]:
for r in graph.run('''
MATCH (c:Character)-[r:INTERACTS]-()
RETURN c.name AS character, sum(r.weight) AS weightedDegree ORDER BY weightedDegree DESC LIMIT 10
'''):
    print(r)

('character': 'Tyrion', 'weightedDegree': 551)
('character': 'Jon', 'weightedDegree': 442)
('character': 'Sansa', 'weightedDegree': 383)
('character': 'Jaime', 'weightedDegree': 372)
('character': 'Bran', 'weightedDegree': 344)
('character': 'Robb', 'weightedDegree': 342)
('character': 'Samwell', 'weightedDegree': 282)
('character': 'Arya', 'weightedDegree': 269)
('character': 'Joffrey', 'weightedDegree': 255)
('character': 'Daenerys', 'weightedDegree': 232)


### Betweenness centrality

In [67]:
for r in graph.run('''
MATCH (c:Character)
WITH collect(c) AS characters
CALL apoc.algo.betweenness(['INTERACTS'], characters, 'BOTH') YIELD node, score
SET node.betweenness = score
RETURN node.name AS name, score ORDER BY score DESC LIMIT 10
'''):
    print(r)

('name': 'Jon', 'score': 1279.7533534055322)
('name': 'Robert', 'score': 1165.6025171231624)
('name': 'Tyrion', 'score': 1101.3849724234349)
('name': 'Daenerys', 'score': 874.8372110508583)
('name': 'Robb', 'score': 706.5572832464792)
('name': 'Sansa', 'score': 705.1985623519137)
('name': 'Stannis', 'score': 571.5247305125714)
('name': 'Jaime', 'score': 556.1852522889822)
('name': 'Arya', 'score': 443.01358430043337)
('name': 'Tywin', 'score': 364.7212195528086)


### Closeness centrality

In [69]:
for r in graph.run('''
MATCH (c:Character)
WITH collect(c) AS characters
CALL apoc.algo.closeness(['INTERACTS'], characters, 'BOTH') YIELD node, score
RETURN node.name AS name, score ORDER BY score DESC LIMIT 10
'''):
    print(r)

('name': 'Tyrion', 'score': 0.004830917874396135)
('name': 'Sansa', 'score': 0.004807692307692308)
('name': 'Robert', 'score': 0.0047169811320754715)
('name': 'Robb', 'score': 0.004608294930875576)
('name': 'Arya', 'score': 0.0045871559633027525)
('name': 'Jaime', 'score': 0.004524886877828055)
('name': 'Stannis', 'score': 0.004524886877828055)
('name': 'Jon', 'score': 0.004524886877828055)
('name': 'Tywin', 'score': 0.004424778761061947)
('name': 'Eddard', 'score': 0.004347826086956522)


## Using python-igraph

### Building an igraph instance from Neo4j

In [11]:
#! pip install python-igraph
from igraph import Graph as IGraph

query = '''
MATCH (c1:Character)-[r:INTERACTS]->(c2:Character)
RETURN c1.name, c2.name, r.weight AS weight
'''

ig = IGraph.TupleList(graph.run(query), weights=True)

ig

### PageRank

![](http://www.lyonwj.com/public/img/page-rank.png)



In [12]:
pg = ig.pagerank()
pgvs = []
for p in zip(ig.vs, pg):
    pgvs.append({"name": p[0]["name"], "pg": p[1]})
print(pgvs[:5])

write_clusters_query = '''
UNWIND {nodes} AS n
MATCH (c:Character) WHERE c.name = n.name
SET c.pagerank = n.pg
'''

graph.run(write_clusters_query, nodes=pgvs)


[{'pg': 0.007328980991947573, 'name': 'Aemon'}, {'pg': 0.02161972592380347, 'name': 'Samwell'}, {'pg': 0.006512533024510501, 'name': 'Grenn'}, {'pg': 0.005477506014302076, 'name': 'Aerys'}, {'pg': 0.022292016521362853, 'name': 'Robert'}]


In [75]:
for r in graph.run('''
MATCH (n:Character)
RETURN n.name AS name, n.pagerank AS pagerank ORDER BY pagerank DESC LIMIT 10
'''):
    print(r)

('name': 'Tyrion', 'pagerank': 0.042884981999963316)
('name': 'Jon', 'pagerank': 0.03582869669163558)
('name': 'Robb', 'pagerank': 0.03017114665594764)
('name': 'Sansa', 'pagerank': 0.030009716660108578)
('name': 'Daenerys', 'pagerank': 0.02881425425830273)
('name': 'Jaime', 'pagerank': 0.028727587587471206)
('name': 'Tywin', 'pagerank': 0.02570016262642541)
('name': 'Robert', 'pagerank': 0.022292016521362864)
('name': 'Cersei', 'pagerank': 0.022287327589773507)
('name': 'Arya', 'pagerank': 0.022050209663844467)


### Community detection

![](http://www.lyonwj.com/public/img/community-1.png)

In [76]:
clusters = IGraph.community_walktrap(ig, weights="weight").as_clustering()

nodes = [{"name": node["name"]} for node in ig.vs]
for node in nodes:
    idx = ig.vs.find(name=node["name"]).index
    node["community"] = clusters.membership[idx]

print(nodes[:5])

write_clusters_query = '''
UNWIND {nodes} AS n
MATCH (c:Character) WHERE c.name = n.name
SET c.community = toInt(n.community)
'''

graph.run(write_clusters_query, nodes=nodes)

[{'community': 0, 'name': 'Aemon'}, {'community': 0, 'name': 'Samwell'}, {'community': 0, 'name': 'Grenn'}, {'community': 1, 'name': 'Aerys'}, {'community': 1, 'name': 'Tywin'}]


In [77]:
for r in graph.run('''
MATCH (c:Character)
WITH c.community AS cluster, collect(c.name) AS  members
RETURN cluster, members ORDER BY cluster ASC
'''):
    print(r)

('cluster': 0, 'members': ['Aemon', 'Alliser', 'Craster', 'Eddison', 'Gilly', 'Janos', 'Jon', 'Mance', 'Rattleshirt', 'Samwell', 'Val', 'Ygritte', 'Grenn', 'Karl', 'Bowen', 'Dalla', 'Orell', 'Qhorin', 'Styr'])
('cluster': 1, 'members': ['Aerys', 'Amory', 'Balon', 'Brienne', 'Bronn', 'Cersei', 'Gregor', 'Jaime', 'Joffrey', 'Jon Arryn', 'Kevan', 'Loras', 'Lysa', 'Meryn', 'Myrcella', 'Oberyn', 'Podrick', 'Renly', 'Robert', 'Robert Arryn', 'Sansa', 'Shae', 'Tommen', 'Tyrion', 'Tywin', 'Varys', 'Walton', 'Petyr', 'Elia', 'Ilyn', 'Pycelle', 'Qyburn', 'Margaery', 'Olenna', 'Marillion', 'Ellaria', 'Mace', 'Chataya', 'Doran'])
('cluster': 2, 'members': ['Arya', 'Beric', 'Eddard', 'Gendry', 'Sandor', 'Anguy', 'Thoros'])
('cluster': 3, 'members': ['Brynden', 'Catelyn', 'Edmure', 'Hoster', 'Lothar', 'Rickard', 'Robb', 'Roose', 'Walder', 'Jeyne', 'Roslin', 'Ramsay'])
('cluster': 4, 'members': ['Bran', 'Hodor', 'Jojen', 'Luwin', 'Meera', 'Rickon', 'Nan', 'Theon'])
('cluster': 5, 'members': ['Belwas'

## Visualization

![](http://www.lyonwj.com/public/img/graph-of-thrones.png)

See [neovis.js](https://github.com/johnymontana/neovis.js)